<a href="https://colab.research.google.com/github/Alexandr20i/Th_coding/blob/main/Th_coding_LR6_Belov_Vinogradov_Grigorev_Semenickhin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Лабораторная работа №6




In [37]:
import numpy as np
import itertools as iter

## 6.1 Написать функции кодирования и декодирования для циклического кода (7,4) с порождающим многочленом 𝑔(𝑥) = 1 + $x^2$ + $x^2$, исправляющего однократные ошибки и провести исследование этого кода для одно-, двух- и трёхкратных ошибок.

In [38]:
def generate_G_matrix_matrix(n, m, g_x) :
    G_matrix = np.zeros((m, n), dtype=int)
    for i in range(m) :
        G_matrix[i, i :  i + g_x.size] = g_x
    for i in range(1, m) :
        for j in range(i) :
            if G_matrix[j][i] == 1 :
                G_matrix[j] ^= G_matrix[i]
    return G_matrix

In [39]:
def vec_to_num(v) :
    return (np.asarray([2 ** i for i in range(v.size)]) * v).sum()

In [40]:
def find_remainder(bin_pol_num_1, bin_pol_num_2, n, m) :
    bin_pol_num_2 <<= m
    for i in range(1, m + 1) :
        bin_pol_num_2 >>= 1
        if bin_pol_num_1 >= 1 << (n - i) :
            bin_pol_num_1 ^= bin_pol_num_2
    return bin_pol_num_1


In [41]:
def deencoded_data_word_data_word(w, m, t, g_x_num) :
    n = w.size
    w_num = vec_to_num(w)
    w_num = find_remainder(w_num, g_x_num, n, m)
    if w_num == 0 :
        return w.copy()[ : m]
    k = 0
    while k > -n :
        if w_num < 1 << t and w_num % 2 == 1 :   #
            break
        w_num <<= 1
        if w_num >= 1 << (n - m) :
            w_num ^= g_x_num
        k -= 1
    w_fix = w.copy()
    while w_num != 0 :
        if w_num % 2 == 1 :
            w_fix[k] ^= 1
        k += 1
        w_num >>= 1
    return w_fix[ : m]

In [42]:
n = 7
m = 4
t = 1
g_x = np.asarray([1, 0, 1, 1])
g_x_num = vec_to_num(g_x)
g_x_num

13

In [43]:
G_matrix = generate_G_matrix_matrix(n, m, g_x)
G_matrix

array([[1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 0, 1, 1, 1],
       [0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 0, 1, 1]])

In [44]:
data_word = np.asarray([1, 0, 0, 1])
v = np.matmul(data_word, G_matrix) % 2
v

array([1, 0, 0, 1, 1, 1, 0])

In [45]:
err2 = np.zeros_like(v, dtype=int)
err2[1] = err2[2] = 1
err3 = err2.copy()
err3[3] = 1

In [46]:
w_err2 = v ^ err2
w_err3 = v ^ err3
w_err1_array = [v.copy() ^ err1 for err1 in np.eye(n, dtype=int)]
w_err1_array

[array([0, 0, 0, 1, 1, 1, 0]),
 array([1, 1, 0, 1, 1, 1, 0]),
 array([1, 0, 1, 1, 1, 1, 0]),
 array([1, 0, 0, 0, 1, 1, 0]),
 array([1, 0, 0, 1, 0, 1, 0]),
 array([1, 0, 0, 1, 1, 0, 0]),
 array([1, 0, 0, 1, 1, 1, 1])]

In [47]:
dec_data_word = deencoded_data_word_data_word(v, m, t, g_x_num)
np.array_equal(dec_data_word, data_word)

True

In [48]:
for w in w_err1_array :
    dec_data_word = deencoded_data_word_data_word(w, m, t, g_x_num)
    print(np.array_equal(dec_data_word, data_word))

True
True
True
True
True
True
True


In [49]:
dec_data_word = deencoded_data_word_data_word(w_err2, m, t, g_x_num)
np.array_equal(dec_data_word, data_word)

False

In [50]:
dec_data_word = deencoded_data_word_data_word(w_err3, m, t, g_x_num)
np.array_equal(dec_data_word, data_word)

False

## 6.2 Написать функции кодирования и декодирования для циклического кода (15,9) с порождающим многочленом 𝑔(𝑥) = 1 + $x^3$ + $x^4$ + $x^5$ + $x^6$,исправляющего пакеты ошибок кратности 3 и провести исследование этого кода для пакетов ошибок длины 1, 2, 3 и 4.
Обратите внимание, что пакет ошибок длины t не означает, что все разряды в пределах этого пакета изменятся (см. лекции).

In [51]:
n = 15
m = 9
t = 3
g_x = np.asarray([1, 1, 1, 1, 0, 0, 1])
g_x_num = vec_to_num(g_x)
g_x_num

79

In [52]:
G_matrix = generate_G_matrix_matrix(n, m, g_x)
G_matrix

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1]])

In [53]:
data_word = np.asarray([1, 1, 1, 0, 0, 0, 0, 0, 0])
v = np.matmul(data_word, G_matrix) % 2
v

array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1])

In [54]:
err2 = np.zeros_like(v, dtype=int)
err2[1] = err2[2] = 1
err3 = err2.copy()
err3[3] = 1
err4 = np.zeros_like(v, dtype=int)
err4[6] = err4[7] = err4[9] = 1

In [55]:
w_err2 = v ^ err2
w_err3 = v ^ err3
w_err4 = v ^ err4
w_err1_array = [v.copy() ^ err1 for err1 in np.eye(n, dtype=int)]
w_err1_array

[array([0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1]),
 array([1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1]),
 array([1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1]),
 array([1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1]),
 array([1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1]),
 array([1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1]),
 array([1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1]),
 array([1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1]),
 array([1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1]),
 array([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1]),
 array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1]),
 array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1]),
 array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1]),
 array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1]),
 array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0])]

In [56]:
dec_data_word = deencoded_data_word_data_word(v, m, t, g_x_num)
np.array_equal(dec_data_word, data_word)

True

In [57]:
for w in w_err1_array :
    dec_data_word = deencoded_data_word_data_word(w, m, t, g_x_num)
    print(np.array_equal(dec_data_word, data_word))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [58]:
dec_data_word = deencoded_data_word_data_word(w_err2, m, t, g_x_num)
np.array_equal(dec_data_word, data_word)

True

In [59]:
dec_data_word = deencoded_data_word_data_word(w_err3, m, t, g_x_num)
np.array_equal(dec_data_word, data_word)

True

In [60]:
dec_data_word = deencoded_data_word_data_word(w_err4, m, t, g_x_num)
np.array_equal(dec_data_word, data_word)

False